In [1]:
import os
import time
import polars as pl
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv('./.env')

True

In [2]:
DB_IP = os.environ.get("DB_IP")
DB_PORT = os.environ.get("DB_PORT")
DB_NAME = os.environ.get("DB_NAME")
DB_USER = os.environ.get("DB_USER")
DB_PASSWORD = os.environ.get("DB_PASSWORD")

url = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_IP}:{DB_PORT}/{DB_NAME}"
engine = create_engine(url)

In [3]:
metrics_with_meta = pl.read_csv("../data/metrics_with_meta.csv")
final_basis_short = pl.read_csv("channel_target_sample.csv")

# fix bags

In [4]:
fill_na_columns = ["forwards", "comments", "paid_reactions", 'sum_standard_reactions', "sum_custom_reactions"]

metrics_with_meta = (
    metrics_with_meta
    .with_columns(
        [
            pl.col(x).fill_null(0).alias(x) 
            for x in fill_na_columns
        ]
    )
)

metrics_with_meta = (
    metrics_with_meta
    .with_columns(
        (
            pl.col("forwards") +
            pl.col("comments") +
            pl.col("paid_reactions") +
            pl.col("sum_standard_reactions") +
            pl.col("sum_custom_reactions")
        ).alias("total_activites")
    )
)

metrics_with_meta.sample()

,id,views,forwards,comments,paid_reactions,standard_reactions,custom_reactions,sum_standard_reactions,sum_custom_reactions,total_sum_reactions,good_finger,heart,fire,fun,bad_finger,total_activites,relative_total_sum_reactions,relative_paid_reactions,relative_good_finger,relative_sum_custom_reactions,relative_forwards,relative_heart,relative_comments,relative_fun,relative_sum_standard_reactions,relative_bad_finger,relative_fire,relative_total_activites,channel_id,post_date
i64,i64,f64,f64,f64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,str
16324765,417423,1520.0,0.0,0.0,0,"""{'🔥': 9, '👍': 6, '❤': 6}""","""{}""",21,0,21,6,6,9,0,0,21.0,0.013816,0.0,0.003947,0.0,0.0,0.003947,0.0,0.0,0.013816,0.0,0.005921,0.0,1531069454,"""2024-08-24 16:05:22+00:00"""


In [5]:
not_features = ["id", "views", "standard_reactions", "custom_reactions", "channel_id", "post_date"]
drop_rel = [x for x in metrics_with_meta.columns if "relative_" in x]

cols = list(set(metrics_with_meta.columns) - set(not_features + drop_rel))

metrics_with_meta = (
    metrics_with_meta
    .with_columns(
        [
            (pl.col(x).fill_null(0) / pl.col("views")).alias(f"relative_{x}") 
            for x in cols
        ]
    )
)

metrics_with_meta.sample()

,id,views,forwards,comments,paid_reactions,standard_reactions,custom_reactions,sum_standard_reactions,sum_custom_reactions,total_sum_reactions,good_finger,heart,fire,fun,bad_finger,total_activites,relative_total_sum_reactions,relative_paid_reactions,relative_good_finger,relative_sum_custom_reactions,relative_forwards,relative_heart,relative_comments,relative_fun,relative_sum_standard_reactions,relative_bad_finger,relative_fire,relative_total_activites,channel_id,post_date,relative_
i64,i64,f64,f64,f64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,str,f64
12695238,2692489,4122.0,1.0,0.0,0,"""{}""","""{}""",0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.000243,0.0,0.0,0.0,0.0,0.0,0.0,0.000243,1203415947,"""2024-07-06 11:45:09+00:00""",3079.873362


# compute aggs 

In [6]:
columns_to_agg = [
    'views', 'forwards', 'comments', "paid_reactions", 
    "sum_standard_reactions", 'sum_custom_reactions', 'total_sum_reactions', 
    'good_finger', 'heart', 'fire', 'fun', 'bad_finger', 'total_activites'
]

metrics_with_meta_agg = (
    metrics_with_meta
    .sort(["channel_id", "post_date"])
    .group_by(["channel_id"])
    .agg([
        pl.col(x)
        .rolling_sum(window_size=50)
        .alias(f"{x}_avg_last_50")
        for x in columns_to_agg
    ])
)

In [7]:
metrics_with_meta_agg = (
    metrics_with_meta
    .sort(["channel_id", "post_date"])
    .hstack(
        metrics_with_meta_agg
        .explode([f"{x}_avg_last_50" for x in columns_to_agg])
        .drop("channel_id")
    )
)

print(metrics_with_meta_agg.shape)
metrics_with_meta_agg.sample()

(17544153, 45)


,id,views,forwards,comments,paid_reactions,standard_reactions,custom_reactions,sum_standard_reactions,sum_custom_reactions,total_sum_reactions,good_finger,heart,fire,fun,bad_finger,total_activites,relative_total_sum_reactions,relative_paid_reactions,relative_good_finger,relative_sum_custom_reactions,relative_forwards,relative_heart,relative_comments,relative_fun,relative_sum_standard_reactions,relative_bad_finger,relative_fire,relative_total_activites,channel_id,post_date,relative_,views_avg_last_50,forwards_avg_last_50,comments_avg_last_50,paid_reactions_avg_last_50,sum_standard_reactions_avg_last_50,sum_custom_reactions_avg_last_50,total_sum_reactions_avg_last_50,good_finger_avg_last_50,heart_avg_last_50,fire_avg_last_50,fun_avg_last_50,bad_finger_avg_last_50,total_activites_avg_last_50
i64,i64,f64,f64,f64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,str,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64
2118974,3783009,224.0,0.0,0.0,0,"""{}""","""{}""",0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1451099382,"""2024-08-28 05:01:17+00:00""",9459.705357,9764.0,16.0,0.0,0,0,0,0,0,0,0,0,0,16.0


In [8]:
avg_last_50 = [f"{x}_avg_last_50" for x in columns_to_agg if "views" not in x ]

metrics_with_meta_agg = (
    metrics_with_meta_agg
    .with_columns(
        [
            (pl.col(x).fill_null(0) / pl.col("views_avg_last_50")).alias(f"relative_{x}") 
            for x in avg_last_50
        ]
    )
)

print(metrics_with_meta_agg.shape)
metrics_with_meta_agg.sample()

(17544153, 57)


,id,views,forwards,comments,paid_reactions,standard_reactions,custom_reactions,sum_standard_reactions,sum_custom_reactions,total_sum_reactions,good_finger,heart,fire,fun,bad_finger,total_activites,relative_total_sum_reactions,relative_paid_reactions,relative_good_finger,relative_sum_custom_reactions,relative_forwards,relative_heart,relative_comments,relative_fun,relative_sum_standard_reactions,relative_bad_finger,relative_fire,relative_total_activites,channel_id,post_date,relative_,views_avg_last_50,forwards_avg_last_50,comments_avg_last_50,paid_reactions_avg_last_50,sum_standard_reactions_avg_last_50,sum_custom_reactions_avg_last_50,total_sum_reactions_avg_last_50,good_finger_avg_last_50,heart_avg_last_50,fire_avg_last_50,fun_avg_last_50,bad_finger_avg_last_50,total_activites_avg_last_50,relative_forwards_avg_last_50,relative_comments_avg_last_50,relative_paid_reactions_avg_last_50,relative_sum_standard_reactions_avg_last_50,relative_sum_custom_reactions_avg_last_50,relative_total_sum_reactions_avg_last_50,relative_good_finger_avg_last_50,relative_heart_avg_last_50,relative_fire_avg_last_50,relative_fun_avg_last_50,relative_bad_finger_avg_last_50,relative_total_activites_avg_last_50
i64,i64,f64,f64,f64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,str,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
507019,2166923,1.0,0.0,0.0,0,"""{}""","""{}""",0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1622576835,"""2024-11-26 02:11:23+00:00""",507019.0,86.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
metrics_with_meta_agg.write_csv("../data/metrics_with_meta_agg.csv")

# join final basis

In [10]:
final_basis = pd.read_sql_table("final_basis", schema="ml_house", con=engine)
final_basis.sample()

,level_0,index,id,raw_text,urls,geo,poll,via_bot_id,via_business_bot_id,cnt_urls,...,web,audio,voice,video,gif,channel_id,post_date,id_channels,participants,about
63704,63917,63708,13558723,Спасательные работы после крушения вертолета п...,nan,None,None,None,None,1,...,False,False,False,True,False,1430620446,2024-05-19 15:36:20+00:00,1430620446,26556,"Главное о войне на Ближнем Востоке: новости, о..."


In [11]:
final_basis.columns

Index(['level_0', 'index', 'id', 'raw_text', 'urls', 'geo', 'poll',
       'via_bot_id', 'via_business_bot_id', 'cnt_urls', 'target_flag1',
       'target_flag2', 'target_flag3', 'target_flag4', 'target_flag5',
       'target_flag6', 'target_flag7', 'target_flag8', 'target_flag9',
       'target_flag10', 'views', 'forwards', 'comments', 'paid_reactions',
       'standard_reactions', 'custom_reactions', 'sum_standard_reactions',
       'sum_custom_reactions', 'total_sum_reactions', 'good_finger', 'heart',
       'fire', 'fun', 'bad_finger', 'total_activites',
       'relative_total_sum_reactions', 'relative_paid_reactions',
       'relative_good_finger', 'relative_sum_custom_reactions',
       'relative_forwards', 'relative_heart', 'relative_comments',
       'relative_fun', 'relative_sum_standard_reactions',
       'relative_bad_finger', 'relative_fire', 'relative_total_activites',
       'is_post', 'silent', 'noforwards', 'pinned', 'fwd_from_flag', 'photo',
       'document', 'web', '

In [12]:
cols = ['id', 'raw_text', 'urls', 'geo', 'poll',
       'via_bot_id', 'via_business_bot_id', 'cnt_urls', 'target_flag1',
       'target_flag2', 'target_flag3', 'target_flag4', 'target_flag5',
       'target_flag6', 'target_flag7', 'target_flag8', 'target_flag9',
       'target_flag10', 'is_post', 'silent', 'noforwards', 'pinned', 'fwd_from_flag', 
       'photo', 'document', 'web', 'audio', 'voice', 'video', 'gif', 'channel_id',
       'post_date', 'id_channels', 'participants', 'about']

In [13]:
final_basis = pl.DataFrame(final_basis[cols])
print(final_basis.shape)
final_basis.sample()

(177373, 35)


id,raw_text,urls,geo,poll,via_bot_id,via_business_bot_id,cnt_urls,target_flag1,target_flag2,target_flag3,target_flag4,target_flag5,target_flag6,target_flag7,target_flag8,target_flag9,target_flag10,is_post,silent,noforwards,pinned,fwd_from_flag,photo,document,web,audio,voice,video,gif,channel_id,post_date,id_channels,participants,about
i64,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,i64,"datetime[ns, UTC]",i64,i64,str
8486938,"""🗣 Материалы #ПерсоналЭкспо2024…","""{https://hrnet.ru/,https://per…",null,null,null,null,3,0,0,0,0,0,0,0,0,1,0,true,false,false,false,false,false,false,false,false,false,false,false,1364654610,2024-08-29 05:54:01 UTC,1364654610,8420,"""Самое большое, взрослое (нам у…"


In [31]:
print(final_basis.shape)

basis_metrics = (
    final_basis
    .join(
        (
            metrics_with_meta_agg
            .drop(pl.col(""))
            .with_columns(
                pl.col("post_date")
                .str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S%z")
                .dt.cast_time_unit("ns")
                .dt.convert_time_zone("UTC")
                )
        ),
        on = ["id", "channel_id", "post_date"],
        how="inner"
    )
)

print(basis_metrics.shape)

(177373, 35)
(177373, 88)


In [32]:
basis_metrics = basis_metrics.to_pandas()

In [33]:
start = time.time() 

with engine.connect() as conn:

    (
        basis_metrics
        .to_sql(name="final_basis_with_metrics", 
                con=conn,
                schema="ml_house",
                if_exists='replace', 
                index=False,
                chunksize=1000
                )
    )
    
print((time.time() - start)/60)

23.04857861995697
